## creating classes and objects

Long term goal: use this notebook at training data to mangle/uglify/transpile any language

demonstrate patterns between different languages, in C# interfaces are like:

```
interface MyInterface {
    myProperty: string
}

class MyClass : MyInterface {
}
```




### applying an interface?


In [ ]:
// in pre-prototype days objects were treated as classes

function applyInterface(class) {
    var properties = ['myProperty'];
    var i, result = {};
    for(i=0; i<properties.length; i++) {
        if(typeof class[properties[i]] === 'undefined') {
            throw new Error(`Missing interface property ${properties[i]}`);
        }
        result[properties[i]] = class[properties[i]];
    }
}



### making a class the normal way?


In [6]:
if(typeof parentClass === 'undefined') {
    class parentClass {
        constructor() {
            console.log('parent');
        }

        do_message() {
            console.log('original');
        }
    }

    class childClass extends parentClass {
        constructor() {
            console.log('child');
            super();
        }

        do_message() {
            super.do_message();
            console.log('overridden');
        }
    }
}

// TODO: better way to do this, childClass.prototype.bind?
module.exports = () => new childClass;

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports);
    var testClass = module.exports();
    testClass.do_message();
    
    /* expected output
    function
    child
    parent
    original
    overridden
    */
    
    /*
    this is extremely annoying, how to achieve the same effect with prototype?
    the javascript language is apparently not indempotent
    
    evalmachine.<anonymous>:1
    class parentClass {
    ^

    SyntaxError: Identifier 'parentClass' has already been declared
        at evalmachine.<anonymous>:1:1
    */
}




function
child
parent
original
overridden


### making a class with prototype

derived from to help avoid annoying notebook errors:

https://stackoverflow.com/questions/10430279/extending-an-object-in-javascript


In [22]:
var parentObj = Object.create({
    do_init() {
        console.log('parent');
    },
    do_message() {
        console.log('original');
    }
})

var childObj = Object.create(parentObj);
Object.assign(childObj, {
    do_init() {
        console.log('child');
        parentObj.do_init();
    },
    do_message() {
        parentObj.do_message();
        console.log('overridden');
    }
})

module.exports = Object.create.bind(null, childObj)

if(typeof $$ !== 'undefined') {
    console.log(typeof module.exports)
    var testClass = module.exports();
    testClass.do_init();
    testClass.do_message();
    
    /* expected output
    child
    parent
    original
    overridden
    */
}



function
child
parent
original
overridden


### enforcing an interface?

test enforcing an interface with prototype?



In [1]:

// or in es6 using expanders

function superInterface(interface) {
}

// inheriting interfaces
function myInterface(interface) {
    return Object.assign({
        myProperty
    } = interface, superInterface(interface));
}

// enforcing interface for use within the caller context
function myInterface(interface) {
    return superInterface(Object.assign({
        myProperty
    } = interface, interface));
}

// doesn't work


child
parent
original
overridden


#### test enforcing an interface?


In [8]:

function myInterface(overrides) {
    var interface = Object.create({});
    Object.assign(interface, {
        propertyOne: overrides.propertyOne,
        print: overrides.print
    });
    return interface;
}

function print() {
    console.log(this.propertyOne);
    console.log(this.propertyTwo);
}

var myClass = Object.create({
    propertyOne: 'original 1',
    propertyTwo: 'original 2',
    print: print
})

function overrideClass(overrides) {
    var overridden = Object.create(myClass);
    Object.assign(overridden, overrides);
    var self = Object.create(overridden);
    return self;
}

myClass.print();
var overridden = overrideClass({
    propertyOne: 'overridden 1',
    propertyTwo: 'overridden 2'
})
overridden.print();
var interfaced = myInterface(overridden);
interfaced.print();
overrideClass(interfaced).print();


/* expected output
original 1
original 2
overridden 1
overridden 2
overridden 1
undefined
overridden 1
original 2
*/



original 1
original 2
overridden 1
overridden 2
overridden 1
undefined
overridden 1
original 2


In [7]:
Object.assign({something: true}, void 0);


{ something: true }